# Advanced Optimizations in Deep Learning

Optimization algorithms play a crucial role in deep learning, helping models learn from data by minimizing or maximizing an objective function. Advanced optimizers like Adam and RMSProp incorporate mechanisms that consider not just the first derivatives of the loss function but also the second moments of the gradients or adaptive learning rates, thereby enhancing the performance and convergence speed of deep neural networks.

https://www.ruder.io/optimizing-gradient-descent/

<img src="./imgs/deep_learning_optimization.jpeg" alt="drawing" width="600"/>

### Stochastic Gradient Descent

Stochastic Gradient Descent (SGD) is the fundamental optimizer in Deep Learning. It iteratively updates the weights of a neural network based on the negative gradient of the loss function. This essentially pushes the weights in a direction that minimizes the loss. However, SGD has limitations:

* **Slow Convergence:** SGD can be slow to converge, especially in landscapes with many shallow valleys (local minima). Imagine a ball rolling down a bumpy hill. It might get stuck in a small valley instead of reaching the global minimum at the bottom.

* **Sensitivity to Learning Rate:** The learning rate hyperparameter controls the step size taken during each update. Too high a learning rate can cause the weights to overshoot the minimum, bouncing around like a ball with too much momentum. Conversely, a very small learning rate makes the journey to the minimum painstakingly slow. 


### RMSProp (Root Mean Squared Prop)

RMSProp addresses SGD's sensitivity to learning rates by introducing an **exponentially decaying average of squared gradients**. This average is used to scale the learning rate for each parameter, leading to **adaptive learning behavior**.

* **Analogy:** Imagine a ball rolling down a hill, but the ground is constantly changing its friction. RMSProp adjusts the ball's speed based on the recent steepness it encountered. 

* **Benefits:**
    * **Faster Convergence:** Compared to SGD's constant learning rate, RMSProp's adaptation often leads to faster convergence.
    * **Handles Non-stationary Problems:** RMSProp can navigate the changing landscape more effectively when working with noisy or non-stationary data.


<img src="./imgs/contour.webp" alt="drawing" width="400"/>


### Adam (Adaptive Moment Estimation)

Adam is an optimizer that combines the strengths of SGD, Momentum, and RMSProp. It maintains an exponentially decaying average of gradients (similar to RMSProp) and another average of squared gradients with a bias correction. 

* **Analogy:** Imagine a ball with built-in momentum rolling on a complex surface. Adam considers both the immediate slope and the historical fluctuations to update the learning rate, leading to efficient navigation.

* **Benefits:**
    * **Often the Best Performer:** Adam is frequently considered the default choice due to its effectiveness across various problems. 
    * **Minimal Hyperparameter Tuning:** Adam generally requires less hyperparameter tuning compared to SGD or RMSprop.
    * **Works Well for Sparse Gradients:** Adam addresses the issue of weights that have very small or zero gradients in certain neural networks. 


![Adam Optimization](imgs/gradient_descent.gif)

### Comparison of Optimizers

To compare the performance of SGD, RMSProp, and Adam optimizers on the MNIST dataset, we'll perform the following steps:

1. Load the MNIST dataset.
2. Preprocess the data.
3. Define a simple neural network model for digit classification.
4. Train the model using each of the three optimizers: SGD, RMSProp, and Adam.
5. Evaluate and compare the performance of the models.


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

#### 1. Load the MNIST Dataset

In [10]:
# Define a transformation to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),  # scales to [0, 1]
])
# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


#### 2. Define the Neural Network Model

In [11]:
# Define the neural network architecture
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.dense_layers = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        return self.dense_layers(x)

#### 3. Train Models with Different Optimizers

In [12]:
# Function to create and compile the model with different optimizers
def create_model(optimizer_name):
    model = NeuralNet()
    if optimizer_name == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=0.01)
    elif optimizer_name == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=0.01)
    elif optimizer_name == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=0.01)
    return model, optimizer

# Function to train and evaluate the model
def train_and_evaluate(model, optimizer, train_loader, test_loader):
    criterion = nn.CrossEntropyLoss()
    for epoch in range(5):
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc = 100. * correct / len(test_loader.dataset)
    return test_loss, test_acc

#### 4. Evaluation and Comparison

In [13]:
# Main experiment loop
optimizers = ['sgd', 'rmsprop', 'adam']
results = {}

for opt in optimizers:
    print(f"Training with {opt.upper()} optimizer...")
    start_time = time.time()
    model, optimizer = create_model(opt)
    test_loss, test_acc = train_and_evaluate(model, optimizer, train_loader, test_loader)
    duration = time.time() - start_time
    results[opt] = (test_loss, test_acc, duration)
    print(f"{opt.upper()} - Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}, Training Time: {duration:.2f} seconds\n")

Training with SGD optimizer...
SGD - Test Loss: 0.2395, Test Accuracy: 93.2400, Training Time: 11.22 seconds

Training with RMSPROP optimizer...
RMSPROP - Test Loss: 0.2502, Test Accuracy: 94.9900, Training Time: 14.49 seconds

Training with ADAM optimizer...
ADAM - Test Loss: 0.2148, Test Accuracy: 95.2500, Training Time: 14.74 seconds



Comparing SGD, RMSProp, and Adam on the MNIST dataset illustrates the impact of advanced optimization techniques in deep learning training. While SGD serves as a solid baseline, optimizers that adjust learning rates adaptively, such as RMSProp and Adam, can significantly enhance model performance. Adam, in particular, stands out for achieving the best balance between high accuracy, low loss, and reasonable training time, making it an excellent choice for many deep learning tasks. 

### Choosing the Right Optimizer

* **There's No Silver Bullet:** The optimal optimizer choice depends on the specific problem, dataset, and model architecture. Experimentation is key!
* **Start with Adam:** Due to its robustness and adaptability, Adam is an excellent starting point for most deep learning tasks.
* **Consider RMSProp for Noisy Data:** If your data exhibits significant noise or non-stationarity, RMSProp might be a better choice than Adam. 
* **Fine-Tuning with SGD:**  In some cases, switching to SGD with a carefully selected learning rate can help refine a model that's already been pre-trained with Adam or RMSprop. 

**Remember:** The best way to determine the most suitable optimizer for your deep learning project is through experimentation! 
